### Importing Libraries

In [1]:
import pandas as pd
from pathlib import Path
import re


In [2]:
### File names specification

ridership = "Metro_Ridership_2025-08-08_00-00-00.csv"
stations  = "Metro_Stations.csv"

In [3]:
### LOADING FILES

txn = pd.read_csv(ridership)
stn = pd.read_csv(stations)
out = OUT = Path("processed"); OUT.mkdir(exist_ok=True)

### OUTPUTTING txn df

txn.head()

,txn_type,txn_date,txn_time,start_location,end_location,line_name,start_zone,end_zone
0,Check out,2025-08-08,07:47:44,Al Nahda Metro Station,Al Nahda Metro Station,Green Metro Line,Zone 5,Zone 5
1,Check out,2025-08-08,17:40:27,Burj Khalifa/ Dubai Mall Metro Station,Burj Khalifa/ Dubai Mall Metro Station,Red Metro Line,Zone 6,Zone 6
2,Check out,2025-08-08,08:28:15,Business Bay Metro Station,Business Bay Metro Station,Red Metro Line,Zone 6,Zone 6
3,Check out,2025-08-08,16:23:42,Emirates Towers Metro Station,Emirates Towers Metro Station,Red Metro Line,Zone 6,Zone 6
4,Check out,2025-08-08,21:36:06,Al Qusais Metro Station,Al Qusais Metro Station,Green Metro Line,Zone 5,Zone 5


In [4]:
### OUTPUTTING stn df

stn.head()

,location_id,zone_id,location_name_english,location_name_arabic,line_name,station_location_longitude,station_location_latitude,station_opening_date,station_closing_date
0,177,6,Dubai Healthcare City Metro Station,مدينة دبي الطبية,Green Metro line,55.322866,25.230903,09/09/2009 00:00:00,NaN
1,168,5,Salah Al Din Metro Station,صلاح الدين,Green Metro line,55.320668,25.270345,09/09/2009 00:00:00,NaN
2,154,2,DMCC Metro Station,مركز دبي للسلع المتعددة,Red Metro line,55.138672,25.070824,09/09/2009 00:00:00,NaN
3,157,2,Energy Metro Station,الطاقة,Red Metro line,55.101247,25.026290,09/09/2009 00:00:00,NaN
4,162,5,Dubai Airport Free Zone Metro Station,المنطقة الحرة بمطار دبي,Green Metro line,55.375009,25.269928,09/09/2009 00:00:00,NaN


In [5]:
### txn Exploration 

txn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1171975 entries, 0 to 1171974
Data columns (total 8 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   txn_type        1171975 non-null  object
 1   txn_date        1171975 non-null  object
 2   txn_time        1171975 non-null  object
 3   start_location  1171975 non-null  object
 4   end_location    1171975 non-null  object
 5   line_name       1171975 non-null  object
 6   start_zone      1171975 non-null  object
 7   end_zone        1171975 non-null  object
dtypes: object(8)
memory usage: 71.5+ MB


In [6]:
###  Parse date and time, build hour

txn["txn_date"]=pd.to_datetime(txn["txn_date"], errors="coerce")
txn["hour"]=pd.to_datetime(txn["txn_time"], format="%H:%M:%S", errors="coerce").dt.hour
txn.head()

,txn_type,txn_date,txn_time,start_location,end_location,line_name,start_zone,end_zone,hour
0,Check out,2025-08-08,07:47:44,Al Nahda Metro Station,Al Nahda Metro Station,Green Metro Line,Zone 5,Zone 5,7
1,Check out,2025-08-08,17:40:27,Burj Khalifa/ Dubai Mall Metro Station,Burj Khalifa/ Dubai Mall Metro Station,Red Metro Line,Zone 6,Zone 6,17
2,Check out,2025-08-08,08:28:15,Business Bay Metro Station,Business Bay Metro Station,Red Metro Line,Zone 6,Zone 6,8
3,Check out,2025-08-08,16:23:42,Emirates Towers Metro Station,Emirates Towers Metro Station,Red Metro Line,Zone 6,Zone 6,16
4,Check out,2025-08-08,21:36:06,Al Qusais Metro Station,Al Qusais Metro Station,Green Metro Line,Zone 5,Zone 5,21


In [7]:
### Hourly profile
hourly = txn.groupby("hour").size().rename("ridership").reset_index().sort_values("hour")
hourly.to_csv(OUT / "hourly.csv", index=False)

In [8]:
### Line split

line_split = txn["line_name"].astype(str).str.strip().str.title().value_counts().reset_index()
line_split.columns = ["line_name", "count"]
line_split.to_csv(OUT / "line-split.csv", index=False)

In [9]:
### Canonical station names for joining to lat/long
def canon(s: str) -> str:
    s = str(s)
    s = s.replace("’","'").replace("`","'")
    s = re.sub(r"\s*/\s*", "/", s)         # normalize slashes
    s = re.sub(r"\s+", " ", s)             # collapse spaces
    s = s.replace(" Metro Station","")     # drop suffix
    return s.strip().lower()

stn["station"] = stn["location_name_english"].map(canon)
txn["start_station"] = txn["start_location"].map(canon)
txn["end_station"]   = txn["end_location"].map(canon)

In [10]:
### Top stations for check in and check out
ins  = txn[txn["txn_type"].str.lower().str.contains("check in", na=False)]
outs = txn[txn["txn_type"].str.lower().str.contains("check out", na=False)]

top_in  = ins["start_station"].value_counts().head(20).reset_index()
top_in.columns = ["station","count"]
top_out = outs["end_station"].value_counts().head(20).reset_index()
top_out.columns = ["station","count"]

In [11]:
### Attach lat/long for mapping
st_geo = stn[["station","station_location_latitude","station_location_longitude"]]
top_in  = top_in.merge(st_geo, on="station", how="left")
top_out = top_out.merge(st_geo, on="station", how="left")

In [12]:
# Save individual and combined lists
top_in.to_csv(OUT / "top-stations-checkin.csv", index=False)
top_out.to_csv(OUT / "top-stations-checkout.csv", index=False)

both = pd.concat(
    [top_in.assign(direction="Check-in"), top_out.assign(direction="Check-out")],
    ignore_index=True
)
both.to_csv(OUT / "top-stations-both.csv", index=False)

In [13]:
### Optional counts by transaction type
tx_type = txn["txn_type"].value_counts().reset_index()
tx_type.columns = ["txn_type","count"]
tx_type.to_csv(OUT / "txn-type-counts.csv", index=False)